In [ ]:
import pandas as pd
from sklearn.metrics import r2_score
import scipy.stats
import sklearn.metrics

from sklearn.ensemble import RandomForestRegressor
import pysam


gene_name = "ZFP57"


# Open the BCF file
vcf_file = pysam.VariantFile("./data/VCF/GTEx_Analysis_2017-06-05_v8_WholeGenomeSeq_838Indiv_Analysis_Freeze_SNPsOnly.bcf.gz", "rb") 

# Iterate over the variants
variant_list = []
for variant in vcf_file:
    variant_list.append(variant.id)
    break
# Close the file
vcf_file.close()

data = sorted(variant.samples.keys())

count = 0
for i in data:
    if i == 'GTEX-1LG7Y':
        print(count)
    count +=1

gene_exp = pd.read_csv("/home/tl688/pitl688/enformer_fine_tuning/data/gtex_eqtl_expression_matrix/Whole_Blood.v8.normalized_expression_genesymbol.csv", sep='\t')

df_gene_test = gene_exp[gene_exp['gene_symbol'].isin([gene_name])]
idx_gene = df_gene_test.index

idx_gene

sample_name = list(variant.samples.keys())

# sample_name

import sklearn.linear_model

gene_exp = pd.read_csv("/home/tl688/pitl688/enformer_fine_tuning/data/gtex_eqtl_expression_matrix/Whole_Blood.v8.normalized_expression_genesymbol.csv", sep='\t')

import torch

data_sample = torch.load(f"/gpfs/radev/project/ying_rex/tl688/seq2cells/tests/{gene_name}_ukbiobert_emb_concat")

data_sample = data_sample.numpy()

In [ ]:
df_waittrain = pd.DataFrame(data_sample)

inter_sample = sorted(set(list(gene_exp.columns[4:])).intersection(set(sample_name)))

df_train_update = df_waittrain
df_train_update.index = inter_sample

df_train_update.index

gene_exp_train = gene_exp[inter_sample]

gene_exp_train

import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, random_state=2024, shuffle=True, )
kf.get_n_splits(df_train_update.index)

In [ ]:
#predict for a single gene
data_list_corr = []
data_list_r2 = []
data_list_mse = []
for i, (train_index, test_index) in enumerate(kf.split(df_train_update.index)):
    train_id = df_train_update.index[train_index]
    test_id = df_train_update.index[test_index]
    np.random.seed(2024)
    subsample = np.random.choice(train_id, int(proportion*len(train_id)), replace=False)
    train_id = subsample

    data_train = df_train_update.loc[train_id].values
    y_train = gene_exp_train.loc[idx_gene,train_id].values[0]

    data_test = df_train_update.loc[test_id].values
    y_test = gene_exp_train.loc[idx_gene,test_id].values[0]

    from sklearn.pipeline import Pipeline
    from sklearn.feature_selection import VarianceThreshold
    from sklearn.linear_model import ElasticNetCV

    model = Pipeline([('elasticnet', ElasticNetCV(random_state=0, max_iter=2000))])
    model.fit(data_train,y_train)

    pred_out = model.predict(data_test)
    corr,pval = scipy.stats.pearsonr(pred_out, y_test)
    mse = sklearn.metrics.mean_squared_error(pred_out, y_test)
    print(corr,r2_score(y_test, pred_out),mse)
    data_list_corr.append(corr)
    data_list_r2.append(r2_score(y_test, pred_out))
    data_list_mse.append(mse)
df = pd.DataFrame()
df['corr'] = data_list_corr
df['r2'] = data_list_r2
df['mse'] = data_list_mse


In [ ]:
#data efficient learning
pred_list = []
for proportion in [0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]:
    data_list_corr = []
    data_list_r2 = []
    data_list_mse = []
    for i, (train_index, test_index) in enumerate(kf.split(df_train_update.index)):
        train_id = df_train_update.index[train_index]
        test_id = df_train_update.index[test_index]
        np.random.seed(2024)
        subsample = np.random.choice(train_id, int(proportion*len(train_id)), replace=False)
        train_id = subsample

        data_train = df_train_update.loc[train_id].values
        y_train = gene_exp_train.loc[idx_gene,train_id].values[0]

        data_test = df_train_update.loc[test_id].values
        y_test = gene_exp_train.loc[idx_gene,test_id].values[0]

        from sklearn.pipeline import Pipeline
        from sklearn.feature_selection import VarianceThreshold
        from sklearn.linear_model import ElasticNetCV

    #     model = Pipeline([('vt', VarianceThreshold()), ('elasticnet', ElasticNetCV(random_state=0, max_iter=2000))])
    #     model = RandomForestRegressor()
        model = Pipeline([('elasticnet', ElasticNetCV(random_state=0, max_iter=2000))])
        model.fit(data_train,y_train)

        pred_out = model.predict(data_test)
        corr,pval = scipy.stats.pearsonr(pred_out, y_test)
        mse = sklearn.metrics.mean_squared_error(pred_out, y_test)
        print(corr,r2_score(y_test, pred_out),mse)
        data_list_corr.append(corr)
    pred_list.append(data_list_corr)

for i in pred_list:
    print(i)





df = pd.DataFrame()
df['corr'] = data_list_corr
df['r2'] = data_list_r2
df['mse'] = data_list_mse

gene_name

print(df['corr'])
